### Another version of the CNN to load files in greyscale before tuning the inception model

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import numpy as np
from sklearn.model_selection import train_test_split

import src.picture_stuff as pix

import os
#import tensorflow as tf
import cPickle as pickle

In [2]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

Using TensorFlow backend.


In [3]:
image_default_size = 299
use_gray = True

###  Start here with import of pickled photo files -
see  
http://localhost:8888/notebooks/CAPSTONE/lego-database/app_stuff/Process_pictures_pickle.ipynb

In [4]:
pkl_path = '../brick_pics_pickled/'

with open(pkl_path + "X.pkl") as f_un:
    X = pickle.load(f_un)

with open(pkl_path + "y_list.pkl") as f_un:
    y_list = pickle.load(f_un)

In [5]:
# Convert saved files to grayscale
if use_gray:
    X = pix.convert_to_gray(X)

In [6]:
print X.shape

(1665, 299, 299, 3)


In [7]:
label_dic,y_out = pix.y_to_hot(y_list)

In [8]:
print label_dic

{0: 2357, 1: 2412, 2: 2413, 3: 2420, 4: 2431, 5: 2450, 6: 2780, 7: 3001, 8: 3002, 9: 3003, 10: 3004, 11: 3005, 12: 3007, 13: 3009, 14: 3010, 15: 3020, 16: 3022, 17: 3023, 18: 3024, 19: 3030, 20: 3031, 21: 3032, 22: 3037, 23: 3039, 24: 3062, 25: 3069, 26: 3070, 27: 3297, 28: 3298, 29: 3308, 30: 3460, 31: 3622, 32: 3660, 33: 3666, 34: 3710, 35: 3795, 36: 3895, 37: 4445, 38: 4743, 39: 4854, 40: 4861, 41: 6141, 42: 6179, 43: 6183, 44: 6215, 45: 30499, 46: 30503, 47: 30504, 48: 41748, 49: 41769, 50: 44237, 51: 48183, 52: 48933, 53: 50304, 54: 50955, 55: 61487}


In [9]:
# current working directory
filepath = os.getcwd()

if use_gray:
    fname = filepath + "/saved_models/label_dic_all-gray.pkl"
else:
    fname = filepath + "/saved_models/label_dic_all.pkl"

with open(fname, 'w') as f:
    pickle.dump(label_dic, f)

### NOTE: IN FINAL VERSION, LET THE WHOLE X AND Y SETS BECOME THE TRAINING SETS, AND WORK ONLY ON NEW PICTURES FOR CLASSIFIER

In [10]:
# TRAINING ON ALL OR DOING TRAIN-TEST SPLIT

#X_train, X_test, y_train, y_test = train_test_split( X, y_out, test_size=0.25, random_state=42)
X_train = X
y_train = y_out

In [11]:
print X_train.shape
print y_train.shape

(1665, 299, 299, 3)
(1665, 56)


In [12]:
#stop here

### OK to here -- we have training data.  Now to the keras stuff
Almost directly from  
https://keras.io/applications/#inceptionv3

In [13]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- one node per class (width of y_hot)
num_classes = y_train.shape[1]
predictions = Dense(num_classes, activation='softmax')(x)

In [14]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [15]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
# NOTE: ran this cell several times
model.fit(X_train,y_train, epochs=10, batch_size=32)

Epoch 1/10
 608/1665 [=========>....................] - ETA: 316s - loss: 5.0221 - acc: 0.0263

KeyboardInterrupt: 

In [ ]:
def save_my_model(save_dir,model_name):
    '''Save current Keras model and weights
       INPUTS: save_dir:   STRING, desired directory
               model_name: STRING, desired file name
       OUTPUTS: none
    '''
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name)
    model.save(model_path)
    print('Saved model at %s ' % model_path)

In [ ]:
# SAVE THE PARTIALLY TRAINED MODEL

save_dir = os.path.join(os.getcwd(), 'saved_models')

if use_gray:
    model_name = 'keras_inception_all_partially_trained_gray.h5'
else:
    model_name = 'keras_inception_all_partially_trained.h5'

save_my_model(save_dir,model_name)

In [ ]:
# Number of model layers:
print len(base_model.layers)

In [ ]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

In [ ]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
              loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
#model.fit_generator(...)

model.fit(X_train,y_train, epochs=10, batch_size=32)

### Once we have a working model, save, evaluate, etc.  

In [ ]:
# SAVE THE MODEL
save_dir = os.path.join(os.getcwd(), 'saved_models')

if use_gray:
    model_name = 'keras_inception_all_trained-gray.h5'
else:
    model_name = 'keras_inception_all_trained.h5'

save_my_model(save_dir,model_name)

In [ ]:
#stop here

In [ ]:
train_more = True
if train_more:
    model.fit(X_train,y_train, epochs=30, batch_size=32)

In [ ]:
if train_more:
    # SAVE THE MODEL AGAIN
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    if use_gray:
        model_name = 'keras_inception_all_re-trained-grey.h5'
    else:
        model_name = 'keras_inception_all_re-trained.h5'
    save_my_model(save_dir,model_name)

In [ ]:
stop here

In [ ]:
# Cheating: testing on training data
model.test_on_batch(X_train,y_train)

In [ ]:
# Cheating: testing on training data
predict_gen = model.predict_on_batch(X_train)

In [ ]:
num_predictions = 10

for predict_index, predicted_y in enumerate(predict_gen):
    # STILL CHEATING
    #actual_label = label_dic[np.argmax(y_test[predict_index])]
    actual_label = label_dic[np.argmax(y_train[predict_index])]
    predicted_label = label_dic[np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

In [ ]:
model.test_on_batch(X_test,y_test)

In [ ]:
predict_gen = model.predict_on_batch(X_test)

In [ ]:
num_predictions = 10
for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = label_dic[np.argmax(y_test[predict_index])]
    predicted_label = label_dic[np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

### PART 4D: make predictions on a single picture / label combination

In [ ]:
reload_model = False
if reload_model:
    from keras.models import load_model
    # reload the most recently saved model and train more
    filepath = '/Users/deborahwilliams/Documents/Galvanize/DSI_class/CAPSTONE/lego-database/'
    fname = filepath + 'saved_models/keras_example_inception_trained-gray.h5'
    model = load_model(fname)

In [ ]:
if reload_model:
    filepath = os.getcwd()
    with open(filepath + "/saved_models/label_dic-gray.pkl") as f_un:
        label_dic = pickle.load(f_un)

In [ ]:
picture_index_lookup = {}
for idx,label in enumerate(y_list):
    picture_index_lookup[label]=idx

In [ ]:
idx0 = 101
small_X_test = X_test[idx0:idx0+1]
small_y_test = [y_test[idx0]]

small_predict_gen = model.predict_on_batch(small_X_test)

small_prediction_list,small_prediction_weights = pix.make_full_prediction_list(
    small_predict_gen,label_dic,n_match=5)

In [ ]:
actual = small_y_test[0]
preds = small_prediction_list[0]

idx_preds = [picture_index_lookup[pred] for pred in preds]
#act_pic = X[picture_index_lookup[label_dic[np.argmax(actual)]]]
act_pic = small_X_test
print "label for this item {}".format(actual)
print "Top 5 predictions ", preds

In [ ]:
fig, ax = plt.subplots(1,figsize=(8,8))

ax1 = plt.subplot2grid((4, 4), (0, 0), colspan=3, rowspan=4)
ax1.imshow(act_pic[0])
ax1.set_title("new brick pic")
ax1.grid(False)
ax1.axis('off')

for idx1 in range(4):
    ax2 = plt.subplot2grid((4, 4), (idx1, 3))
    ax2.imshow(X[idx_preds[idx1]])
    ax2.set_title("predict #{}, wt {:.3f}".format(idx1,small_prediction_weights[0][idx1]))
    ax2.grid(False)
    ax2.axis('off')
#fig.savefig("temp.png")